In [92]:
import sys
sys.path.append('../')

from IPython.core.display import display
import numpy as np
import pandas as pd

In [93]:
df = pd.read_csv("./movie_ratings/movie_ratings.csv")

In [94]:
display(df.head())
display(df.describe())

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


,userId,movieId,rating,timestamp
count,100004.000000,100004.000000,100004.000000,1.000040e+05
mean,347.011310,12548.664363,3.543608,1.129639e+09
std,195.163838,26369.198969,1.058064,1.916858e+08
min,1.000000,1.000000,0.500000,7.896520e+08
25%,182.000000,1028.000000,3.000000,9.658478e+08
50%,367.000000,2406.500000,4.000000,1.110422e+09
75%,520.000000,5418.000000,4.000000,1.296192e+09
max,671.000000,163949.000000,5.000000,1.476641e+09


In [95]:
# mark users and movies
df['userId'] = 'u' + df['userId'].astype(str)
df['movieId'] = 'm' + df['movieId'].astype(str)

display(df.head())

,userId,movieId,rating,timestamp
0,u1,m31,2.5,1260759144
1,u1,m1029,3.0,1260759179
2,u1,m1061,3.0,1260759182
3,u1,m1129,2.0,1260759185
4,u1,m1172,4.0,1260759205


In [96]:
# filter to likes (rating ≥ 4)
df = df[df.rating >= 4]

In [97]:
display(df.head())
display(df.userId.value_counts())

,userId,movieId,rating,timestamp
4,u1,m1172,4.0,1260759205
12,u1,m1953,4.0,1260759191
13,u1,m2105,4.0,1260759139
20,u2,m10,4.0,835355493
21,u2,m17,5.0,835355681


u564    1115
u547    1031
u30      702
u73      617
u472     544
        ... 
u429       2
u170       2
u79        1
u579       1
u221       1
Name: userId, Length: 671, dtype: int64

In [98]:
# get max_rating for user
df = df.join(df.groupby('userId')['rating'].max().rename('user_max_rating'), on='userId')
df

,userId,movieId,rating,timestamp,user_max_rating
4,u1,m1172,4.0,1260759205,4.0
12,u1,m1953,4.0,1260759191,4.0
13,u1,m2105,4.0,1260759139,4.0
20,u2,m10,4.0,835355493,5.0
21,u2,m17,5.0,835355681,5.0
...,...,...,...,...,...
99995,u671,m5989,4.0,1064890625,5.0
99996,u671,m5991,4.5,1064245387,5.0
99997,u671,m5995,4.0,1066793014,5.0
100000,u671,m6269,4.0,1065149201,5.0


In [99]:
# for each user get min movieId with highest rating => testMovies
testMovies = df[df['rating'] == df['user_max_rating']].groupby('userId')['movieId'].min().rename('test_movie')
# mark test movies
df = df.join(testMovies, on='userId')
df['is_test'] = df['movieId'] == df['test_movie']
# drop tmp columns
df.drop(['user_max_rating', 'test_movie'], axis=1, inplace=True)

df

,userId,movieId,rating,timestamp,is_test
4,u1,m1172,4.0,1260759205,True
12,u1,m1953,4.0,1260759191,False
13,u1,m2105,4.0,1260759139,False
20,u2,m10,4.0,835355493,False
21,u2,m17,5.0,835355681,False
...,...,...,...,...,...
99995,u671,m5989,4.0,1064890625,False
99996,u671,m5991,4.5,1064245387,False
99997,u671,m5995,4.0,1066793014,False
100000,u671,m6269,4.0,1065149201,False


In [100]:
train = df[df['is_test'] == False]
display(train.head())
display(train.describe())

,userId,movieId,rating,timestamp,is_test
12,u1,m1953,4.0,1260759191,False
13,u1,m2105,4.0,1260759139,False
20,u2,m10,4.0,835355493,False
21,u2,m17,5.0,835355681,False
22,u2,m39,5.0,835355604,False


,rating,timestamp
count,50897.000000,5.089700e+04
mean,4.359589,1.117952e+09
std,0.438699,1.917319e+08
min,4.000000,7.896520e+08
25%,4.000000,9.582482e+08
50%,4.000000,1.094221e+09
75%,5.000000,1.274150e+09
max,5.000000,1.476641e+09


In [101]:
test = df[df['is_test']]
display(test.head())
display(test.describe())

,userId,movieId,rating,timestamp,is_test
4,u1,m1172,4.0,1260759205,True
29,u2,m150,5.0,835355395,True
113,u3,m1197,5.0,1298932770,True
187,u4,m1022,5.0,949949638,True
374,u5,m1035,5.0,1163373103,True


,rating,timestamp
count,671.000000,6.710000e+02
mean,4.975410,1.102036e+09
std,0.127222,2.116010e+08
min,4.000000,8.282124e+08
25%,5.000000,9.390214e+08
50%,5.000000,1.064891e+09
75%,5.000000,1.284735e+09
max,5.000000,1.476478e+09


In [102]:
# write train/test datasets
train.to_csv("../data/movie_ratings/movie_ratings.csv")
train.to_csv("./movie_ratings/movie_ratings_train.csv")
test.to_csv("./movie_ratings/movie_ratings_test.csv")

print("done")

done
